# Aufgabe 9.1

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 21 13:50:37 2021

@author: Chantal
"""
#import os
import pandas as pd
import statsmodels.api as sm
import scipy.stats

In [3]:
# Auslesen der Daten
data = pd.read_excel("weeksworkedv2.xlsx","Sheet1")

# Abhängige Variablen und Regressoren
y = data[['weeks']]
x = data[['morekids', 'educ']]
x = pd.DataFrame(sm.add_constant(x.values, has_constant='add'), \
                    columns = ['Konstante'] + x.columns.tolist())

# Schätzung
weeksreg = sm.OLS(y,x).fit()

# Berechnung der Residuen des geschätzten Modells und Quadrierung der Residuen
resid=weeksreg.resid
residsq=resid*resid
#type(resid)
type(residsq)

pandas.core.series.Series

In [4]:
# Quadrierung der unabhängigen Variablen educ von Modell 1 (morekids als Dummy-Variable
# wird nicht quadriert)
data['educsq'] = data['educ']*data['educ']

# Definiere die Interaktion der beiden unabhängigen Variablen von Modell 1
data['educ_morekids'] = data['educ']*data['morekids']

# Festsetzen der unabhängigen Variablen der White-Hilfsregression. Die Hilfsregression
# enthält immer die beiden Regressoren des ursprünglichen Modells, die beiden quadrierten
# Regressoren des ursprünglichen Modells und die Interaktion der beiden Regressoren
XVarWhite = data[['educ','morekids','educsq','educ_morekids']]
XVarWhite = pd.DataFrame(sm.add_constant(XVarWhite.values, has_constant='add'),\
             columns = ['Konstante'] + XVarWhite.columns.tolist())

# Festsetzen der abhängigen Variable der Hilfsregression, hier also die quadrierten
# Residuen von weeksreg
YVarWhite=residsq

# Schätzung der White-Hilfsregession
weeks_White = sm.OLS(YVarWhite,XVarWhite).fit()

## b)


In [5]:
# Berechnung des realisierten Wertes des White-Testes anhand der Anzahl der Beobachtungen
# der Hilfsregression und des Bestimmtheitsmaßes der Hilfsregression
R2_White=weeks_White.rsquared
#k_White= weeks_White.df_model+weeks_White.k_constant
#n_White=int(weeks_White.df_resid +k_White )
White_act=weeks_White.nobs*R2_White
#print("R2: ",R2_White)
#print("n_white: ", n_White)
print('Realisierter Wert zum White-Test', White_act)

Realisierter Wert zum White-Test 3.4980967004657


In [6]:
print(weeks_White.df_model)
print(weeks_White.k_constant)
print(weeks_White.nobs)
print(weeks_White.df_resid)

4.0
1
273.0
268.0



## c)


In [7]:
# Berechnung des kritischen Wertes anhand der Chi-Verteilung zum Signifikanzniveau 1%
#und der Anzahl der Freiheitsgrade k_White-1, wobei k_White für die Anzahl der geschätzten
#Koeffizienten der Hilfsregression steht.
alphaWhite=0.01
chi_krit=scipy.stats.chi2.ppf((1-alphaWhite),weeks_White.df_model)
print('Kritischer Wert zum White-Test', chi_krit)

Kritischer Wert zum White-Test 13.276704135987622


# Aufgabe 9.3

In [40]:
# Auslesen der Daten
data = pd.read_excel("trmgpa.xlsx","Sheet1")

# Abhängige Variablen und Regressoren
y = data[['trmgpa']]
x = data[['female', 'frstsem']]
x = pd.DataFrame(sm.add_constant(x.values, has_constant='add'), \
                    columns = ['const'] + x.columns.tolist())

# Schätzung
trmgpa = sm.OLS(y,x).fit()
#print(trmgpa.summary())
print('Geschätzte Koeffizienten zu trmgpa \n', trmgpa.params)
print('Standardfehler zu trmgpa \n', trmgpa.bse)
print('Robuste Standardfehler zu trmgpa \n', trmgpa.HC0_se)


Geschätzte Koeffizienten zu trmgpa 
 const      2.211046
female     0.569442
frstsem   -0.157168
dtype: float64
Standardfehler zu trmgpa 
 const      0.032019
female     0.061642
frstsem    0.078291
dtype: float64
Robuste Standardfehler zu trmgpa 
 const      0.032990
female     0.055841
frstsem    0.083309
dtype: float64


# Contoh WLS

In [39]:
import numpy as np

nsample = 50
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, (x - 5) ** 2))
#print(X)
X = sm.add_constant(X)
print(X.shape)
beta = [5.0, 0.5, -0.01]
sig = 0.5
w = np.ones(nsample)
print(w)
w[nsample * 6 // 10 :] = 3
#w[ :5] = 3
print(w.shape)
print("Matrix: ",X.shape, " dot ", len(beta))
y_true = np.dot(X, beta)
print(y_true.shape)
e = np.random.normal(size=nsample)
y = y_true + sig * w * e
X = X[:, [0, 1]]

(50, 3)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
(50,)
Matrix:  (50, 3)  dot  3
(50,)


In [23]:
#print(X)
#print(X)
X

array([[ 1.        ,  0.        ],
       [ 1.        ,  0.40816327],
       [ 1.        ,  0.81632653],
       [ 1.        ,  1.2244898 ],
       [ 1.        ,  1.63265306],
       [ 1.        ,  2.04081633],
       [ 1.        ,  2.44897959],
       [ 1.        ,  2.85714286],
       [ 1.        ,  3.26530612],
       [ 1.        ,  3.67346939],
       [ 1.        ,  4.08163265],
       [ 1.        ,  4.48979592],
       [ 1.        ,  4.89795918],
       [ 1.        ,  5.30612245],
       [ 1.        ,  5.71428571],
       [ 1.        ,  6.12244898],
       [ 1.        ,  6.53061224],
       [ 1.        ,  6.93877551],
       [ 1.        ,  7.34693878],
       [ 1.        ,  7.75510204],
       [ 1.        ,  8.16326531],
       [ 1.        ,  8.57142857],
       [ 1.        ,  8.97959184],
       [ 1.        ,  9.3877551 ],
       [ 1.        ,  9.79591837],
       [ 1.        , 10.20408163],
       [ 1.        , 10.6122449 ],
       [ 1.        , 11.02040816],
       [ 1.        ,